In [737]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [738]:
C=70
downweight=0.8

In [739]:
downweight_granularity=['dtime','Direction','dday','month']

# Yield computation

In [740]:
Prediction_pax=pd.read_csv('R_Output_Training_Pax.csv',sep=',')
Prediction_pax=Prediction_pax[['Dprio']+downweight_granularity+['forecast_bookings']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)
Prediction_pax.index=range(Prediction_pax.shape[0])

Prediction_revenue=pd.read_csv('R_Output_Training_Revenue.csv',sep=',')
Prediction_revenue=Prediction_revenue.rename(columns={'forecast_bookings': 'forecast_revenue'})
Prediction_revenue=Prediction_revenue[['Dprio']+downweight_granularity+['forecast_revenue']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)
Prediction_revenue.index=range(Prediction_revenue.shape[0])

Prediction_pax=Prediction_pax.sort_values(by=downweight_granularity+['Dprio'])
Prediction_revenue=Prediction_revenue.sort_values(by=downweight_granularity+['Dprio'])

Prediction=pd.concat([Prediction_pax[[x for x in Prediction_pax.columns if x!='NumPax' and x!='forecast_node']],
                      Prediction_revenue['forecast_revenue']],axis=1)

Prediction['forecast_yield']=Prediction['forecast_revenue']/Prediction['forecast_bookings']

Prediction.index=range(Prediction.shape[0])

In [741]:
Prediction=Prediction.set_index(['Dprio']+downweight_granularity)

Cumulation_forecast_bookings=Prediction.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_bookings'].cumsum().reset_index()

Cumulation_forecast_revenue=Prediction.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_revenue'].cumsum().reset_index()

Cumulation_forecast=Cumulation_forecast_bookings.merge(Cumulation_forecast_revenue,on=['Dprio']+downweight_granularity)

Prediction=Prediction.reset_index()

In [742]:
C_datadriven=pd.read_csv('R_Training_Pax.csv')
C_datadriven=C_datadriven.groupby(['DepDate']+downweight_granularity)['NumPax'].sum().reset_index()
C_datadriven=C_datadriven.groupby(downweight_granularity)['NumPax'].max().reset_index().rename(columns={'NumPax': 'downweight'})
C_datadriven['downweight']=C_datadriven['downweight'].astype('float')/C
C_datadriven['downweight']=C_datadriven['downweight'].apply(lambda x: downweight if x>downweight else x)
# C_datadriven['downweight']=downweight

C_datadriven['C']=C
C_datadriven['C_downweighted']=C*C_datadriven['downweight']

C_datadriven=C_datadriven[[x for x in C_datadriven.columns if x!='downweight']]

###########################################################

# MaxAttainable=Prediction.merge(C_datadriven,on=downweight_granularity)
# MaxAttainable['forecast_bookings_sum']=MaxAttainable.groupby(downweight_granularity)['forecast_bookings'].transform('sum')

# MaxAttainable['forecast_bookings_stand_full']=MaxAttainable['C']*(MaxAttainable['forecast_bookings']/MaxAttainable['forecast_bookings_sum'])
# MaxAttainable['forecast_bookings_stand_downweighted']=MaxAttainable['C_downweighted']*(MaxAttainable['forecast_bookings']/MaxAttainable['forecast_bookings_sum'])
# MaxAttainable['revenue_full']=MaxAttainable['forecast_bookings_stand_full']*MaxAttainable['forecast_yield']
# MaxAttainable['revenue_downweighted']=MaxAttainable['forecast_bookings_stand_downweighted']*MaxAttainable['forecast_yield']

# MaxAttainable['revenue_full']=MaxAttainable.groupby(downweight_granularity)['revenue_full'].transform('sum')
# MaxAttainable['revenue_downweighted']=MaxAttainable.groupby(downweight_granularity)['revenue_downweighted'].transform('sum')
# MaxAttainable=MaxAttainable[downweight_granularity+['C','C_downweighted','revenue_full','revenue_downweighted']]

# MaxAttainable=MaxAttainable.drop_duplicates(subset=downweight_granularity)

# MaxAttainable.index=range(MaxAttainable.shape[0])

In [743]:
ActualBookings=pd.read_csv('R_Test_Pax.csv')
ActualBookings=ActualBookings[['DepDate','Dprio']+downweight_granularity+['NumPax']]
ActualBookings['Dprio']=-ActualBookings['Dprio']
ActualBookings=ActualBookings.set_index(['Dprio']+downweight_granularity+['DepDate'])
ActualBookings=ActualBookings.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['NumPax'].cumsum().reset_index()
ActualBookings['Dprio']=-ActualBookings['Dprio']

In [744]:
# ActualBookings=ActualBookings.merge(C_datadriven,on=downweight_granularity)
# ActualBookings['PaxNeeded_full']=ActualBookings['C']-ActualBookings['NumPax']
# ActualBookings['PaxNeeded_downweighted']=ActualBookings['C_downweighted']-ActualBookings['NumPax']
# ActualBookings=ActualBookings[[x for x in ActualBookings if x!='NumPax' and x!='C' and x!='C_downweighted']]

ActualRevenue=pd.read_csv('R_Test_Revenue.csv')
ActualRevenue=ActualRevenue[['DepDate','Dprio']+downweight_granularity+['Coupons+YQ on current bookings']]
ActualRevenue['Dprio']=-ActualRevenue['Dprio']
ActualRevenue=ActualRevenue.set_index(['Dprio']+downweight_granularity+['DepDate'])
ActualRevenue=ActualRevenue.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['Coupons+YQ on current bookings'].cumsum().reset_index()
ActualRevenue['Dprio']=-ActualRevenue['Dprio']

In [745]:
Data=ActualBookings.merge(ActualRevenue,on=['DepDate','Dprio']+downweight_granularity)
Data=Data.merge(C_datadriven,on=downweight_granularity)
Data=Data.merge(Cumulation_forecast,on=['Dprio']+downweight_granularity)
Data['revenue_curve_full']=(Data['C']-Data['NumPax'])*(Data['forecast_revenue']/Data['forecast_bookings'])
Data['revenue_curve_downweighted']=(Data['C_downweighted']-Data['NumPax'])*(Data['forecast_revenue']/Data['forecast_bookings'])

# Data['revenue_curve_full']=Data['NumPax']*(Data['forecast_revenue']/Data['forecast_bookings'])
# Data['revenue_curve_downweighted']=Data['NumPax']*(Data['forecast_revenue']/Data['forecast_bookings'])

In [747]:
Data

,Dprio,dtime,Direction,dday,month,DepDate,NumPax,Coupons+YQ on current bookings,C,C_downweighted,forecast_bookings,forecast_revenue,revenue_curve_full,revenue_curve_downweighted
0,365.0,405.0,O,Friday,4,2019-04-12,0.0,0.0,70,49.0,33.774194,3136.516129,6500.706691,4550.494684
1,365.0,405.0,O,Friday,4,2019-04-19,0.0,0.0,70,49.0,33.774194,3136.516129,6500.706691,4550.494684
2,365.0,405.0,O,Friday,4,2019-04-26,0.0,0.0,70,49.0,33.774194,3136.516129,6500.706691,4550.494684
3,364.0,405.0,O,Friday,4,2019-04-12,0.0,0.0,70,49.0,33.772002,3136.482109,6501.058182,4550.740728
4,364.0,405.0,O,Friday,4,2019-04-19,0.0,0.0,70,49.0,33.772002,3136.482109,6501.058182,4550.740728
5,364.0,405.0,O,Friday,4,2019-04-26,0.0,0.0,70,49.0,33.772002,3136.482109,6501.058182,4550.740728
6,363.0,405.0,O,Friday,4,2019-04-12,0.0,0.0,70,49.0,33.769776,3136.447376,6501.414592,4550.990214
7,363.0,405.0,O,Friday,4,2019-04-19,0.0,0.0,70,49.0,33.769776,3136.447376,6501.414592,4550.990214
8,363.0,405.0,O,Friday,4,2019-04-26,0.0,0.0,70,49.0,33.769776,3136.447376,6501.414592,4550.990214
9,362.0,405.0,O,Friday,4,2019-04-12,0.0,0.0,70,49.0,33.767517,3136.411916,6501.775986,4551.243190


In [718]:
#Data['revenue_full']-
#Data['revenue_downweighted']-

In [733]:
Data_365Only=Data.loc[Data['Dprio']==365,:]
Data_365Only.index=range(Data_365Only.shape[0])

In [736]:
test=Data_365Only['C']*(Data_365Only['forecast_revenue']/Data_365Only['forecast_bookings'])

In [ ]:


IdealCurve_average_full=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_full'].cumsum().reset_index().rename(columns={'forecast_bookings_full': 'Ideal_average_full'})
IdealCurve_average_downweighted=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_downweighted'].cumsum().reset_index().rename(columns={'forecast_bookings_downweighted': 'Ideal_average_downweighted'})


In [5]:
IdealCurve=IdealCurve.sort_values(by=downweight_granularity+['Dprio'])
Prediction=Prediction.sort_values(by=downweight_granularity+['Dprio'])

In [6]:
IdealCurve=pd.concat([IdealCurve,Prediction['forecast_yield']],axis=1)

In [7]:
IdealCurve['forecast_revenue_full']=IdealCurve['forecast_bookings_full']*IdealCurve['forecast_yield']
IdealCurve['forecast_revenue_downweighted']=IdealCurve['forecast_bookings_downweighted']*IdealCurve['forecast_yield']

In [8]:
IdealCurve=IdealCurve[downweight_granularity+['Dprio','forecast_revenue_full','forecast_revenue_downweighted']]

In [9]:
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity)
IdealCurve_cumul_full=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_revenue_full'].cumsum().reset_index()
IdealCurve_cumul_full['Dprio']=-IdealCurve_cumul_full['Dprio']

In [10]:
IdealCurve=IdealCurve.reset_index()
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity)
IdealCurve_cumul_downweighted=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_revenue_downweighted'].cumsum().reset_index()
IdealCurve_cumul_downweighted['Dprio']=-IdealCurve_cumul_downweighted['Dprio']

In [11]:
Data=pd.read_csv('R_Output_Test_Revenue.csv',sep=',')

In [12]:
Data=Data[['DepDate','yday','dtime','Direction','month','dday','Dprio','Coupons.YQ.on.current.bookings']]

Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

Data=Data.loc[Data['TicketDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

Data=Data[[x for x in Data.columns if x!='TicketDate']]

Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6]).sum().groupby(level=[0,1,2,3,4,5])['Coupons.YQ.on.current.bookings'].cumsum().reset_index()

Data['Dprio']=-Data['Dprio']

In [13]:
Data=Data.rename(columns={'Coupons.YQ.on.current.bookings': 'ActualRevenue'})

In [14]:
Data=Data.merge(IdealCurve_cumul_full,on=downweight_granularity+['Dprio'])
Data=Data.merge(IdealCurve_cumul_downweighted,on=downweight_granularity+['Dprio'])

In [15]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Data['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Data['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Data=Data.merge(Map_DepTime,on='dtime')

Data['dtime']=Data['deptime']
Data=Data[[x for x in Data.columns if x!='deptime']]

In [16]:
Data.to_csv('GUI_in3.csv',index=False)

In [17]:
Data.loc[Data['DepDate']=='2019-04-15',:]

,DepDate,yday,dtime,Direction,month,dday,Dprio,ActualRevenue,forecast_revenue_full,forecast_revenue_downweighted
39978,2019-04-15,105,14:00,O,4,Monday,365,0,0.000636,0.000509
39981,2019-04-15,105,14:00,O,4,Monday,364,0,0.001295,0.001036
39984,2019-04-15,105,14:00,O,4,Monday,363,0,0.001977,0.001582
39987,2019-04-15,105,14:00,O,4,Monday,362,0,0.002684,0.002147
39990,2019-04-15,105,14:00,O,4,Monday,361,0,0.003415,0.002732
39993,2019-04-15,105,14:00,O,4,Monday,360,0,0.004173,0.003338
39996,2019-04-15,105,14:00,O,4,Monday,359,0,0.004958,0.003966
39999,2019-04-15,105,14:00,O,4,Monday,358,0,0.005770,0.004616
40002,2019-04-15,105,14:00,O,4,Monday,357,0,0.006611,0.005289
40005,2019-04-15,105,14:00,O,4,Monday,356,0,0.007482,0.005986
